<a href="https://colab.research.google.com/github/ramesitexp/NLP_DataScience/blob/main/transformer_tamil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

data_ta = {
    "text": [
        "இந்த ஆப் சரியாக வேலை செய்யவில்லை",
        "வாடிக்கையாளர் சேவை மிகவும் உதவியாக இருந்தது",
        "மிகவும் மோசமான அனுபவம்",
        "புதிய அப்டேட் மிகவும் பிடித்தது",
        "சேவையில் திருப்தி இல்லை",
        "தயாரிப்பு தரம் அருமை",
        "சப்போர்ட் குழு பதில் தரவில்லை",
        "இந்த வாங்குதல் எனக்கு மிகவும் சந்தோஷம்"
    ],
    "label": [0, 1, 0, 1, 0, 1, 0, 1]
}

df = pd.DataFrame(data_ta)
df


,text,label
0,இந்த ஆப் சரியாக வேலை செய்யவில்லை,0
1,வாடிக்கையாளர் சேவை மிகவும் உதவியாக இருந்தது,1
2,மிகவும் மோசமான அனுபவம்,0
3,புதிய அப்டேட் மிகவும் பிடித்தது,1
4,சேவையில் திருப்தி இல்லை,0
5,தயாரிப்பு தரம் அருமை,1
6,சப்போர்ட் குழு பதில் தரவில்லை,0
7,இந்த வாங்குதல் எனக்கு மிகவும் சந்தோஷம்,1


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["label"], test_size=0.25, random_state=42
)

vectorizer = TfidfVectorizer(
    analyzer="word",
    ngram_range=(1,2)   # 🔥 helps Tamil a LOT
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

ml_model = LogisticRegression()
ml_model.fit(X_train_vec, y_train)

pred = ml_model.predict(X_test_vec)

print("Tamil ML Accuracy:", accuracy_score(y_test, pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, pred))


Tamil ML Accuracy: 0.0
Confusion Matrix:
 [[0 0]
 [2 0]]


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(
    num_words=5000,
    oov_token="<OOV>"
)

tokenizer.fit_on_texts(df["text"])

sequences = tokenizer.texts_to_sequences(df["text"])
X_dl = pad_sequences(sequences, maxlen=12)
y_dl = df["label"].values


In [4]:
from sklearn.model_selection import train_test_split

X_train_dl, X_test_dl, y_train_dl, y_test_dl = train_test_split(
    X_dl, y_dl, test_size=0.25, random_state=42
)

dl_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(5000, 64, input_length=12),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

dl_model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

dl_model.fit(
    X_train_dl,
    y_train_dl,
    epochs=10,
    batch_size=2,
    verbose=1
)

loss, acc = dl_model.evaluate(X_test_dl, y_test_dl)
print("Tamil DL Accuracy:", acc)


Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.5000 - loss: 0.6941
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7708 - loss: 0.6786
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6458 - loss: 0.6677
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7708 - loss: 0.6373
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5833 - loss: 0.6532
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7708 - loss: 0.5895
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6458 - loss: 0.6041
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6458 - loss: 0.5972
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6458 - loss: 0.5865
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7708 - loss: 0.4723
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step - accuracy: 0.0000e+00 - loss: 1.3126
Tamil DL Accuracy: 0.0


In [7]:
!pip -q install transformers sentencepiece


In [8]:
from transformers import pipeline

classifier = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-xlm-roberta-base-sentiment"
)


OSError: cardiffnlp/twitter-xlm-roberta-base-sentiment is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

In [ ]:
texts = df["text"].tolist()

results = classifier(texts)

for t, r in zip(texts, results):
    print(f"{t} --> {r}")


இந்த ஆப் சரியாக வேலை செய்யவில்லை --> {'label': 'negative', 'score': 0.7154659628868103}
வாடிக்கையாளர் சேவை மிகவும் உதவியாக இருந்தது --> {'label': 'positive', 'score': 0.5434223413467407}
மிகவும் மோசமான அனுபவம் --> {'label': 'negative', 'score': 0.9400252103805542}
புதிய அப்டேட் மிகவும் பிடித்தது --> {'label': 'positive', 'score': 0.8371915817260742}
சேவையில் திருப்தி இல்லை --> {'label': 'negative', 'score': 0.6618854403495789}
தயாரிப்பு தரம் அருமை --> {'label': 'positive', 'score': 0.6972827911376953}
சப்போர்ட் குழு பதில் தரவில்லை --> {'label': 'neutral', 'score': 0.7073349952697754}
இந்த வாங்குதல் எனக்கு மிகவும் சந்தோஷம் --> {'label': 'positive', 'score': 0.8307168483734131}
